In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from datetime import datetime

# Load CSV data
# def load_data(data_folder):
#     dataframes = []
#     for filename in os.listdir(data_folder):
#         if filename.endswith(".csv"):
#             filepath = os.path.join(data_folder, filename)
#             df = pd.read_csv(filepath)
#             dataframes.append(df)
#     return pd.concat(dataframes)

# Example folder path
data_folder = '../NASAResources/space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-01-19HR00_evid00002.csv'
# Load the catalog (starting points of quakes)
catalog_path = '../NASAResources/space_apps_2024_seismic_detection/data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv'
catalog = pd.read_csv(catalog_path)

# Load training data
#data = load_data(data_folder)


# Delete this
data = pd.read_csv(data_folder)

In [2]:


# Function to reformat the absolute time by setting the fractional seconds to .00000
def reformat_abs_time(time_str):
    dt = datetime.strptime(time_str, '%Y-%m-%dT%H:%M:%S.%f')
    reformatted_time = dt.strftime('%Y-%m-%dT%H:%M:00') + '.000000'
    return reformatted_time


# Apply the reformatting function to the 'time_abs' column
data['time_abs'] = data['time_abs(%Y-%m-%dT%H:%M:%S.%f)'].apply(reformat_abs_time)

# Processing data folder
data['time_rel'] = data['time_rel(sec)'].astype(int)
data['velocity'] = data['velocity(m/s)']

features = data[['time_rel', 'velocity', 'time_abs']]

catalog['time_rel'] = catalog['time_rel(sec)'].astype(int)
catalog['time_abs'] = catalog['time_abs(%Y-%m-%dT%H:%M:%S.%f)']





In [3]:
# Print the first absolute time found in the data DataFrame
print(data['time_rel'].iloc[486944])

73500


In [4]:


# def label_data(data, catalog):
#     # Convert catalog time_rel to a numpy array for fast lookups
#     quake_start_times = catalog['time_rel'].to_numpy()
    
#     # Create an array of labels
#     labels = np.isin(data['time_rel'].to_numpy(), quake_start_times).astype(int)
#     return labels

#     # Apply labeling
# labels = label_data(data, catalog)



In [5]:
#Numpy label creation, should be faster

# def label_data(data, catalog):
#     # Convert catalog time_rel to a numpy array for fast lookups
#     quake_start_times = catalog['time_rel'].to_numpy()

#     # Create an array of labels
#     labels_rel = np.isin(data['time_rel'].to_numpy(), quake_start_times).astype(int)
#     # if np.where(labels == 1):
#     #     quake_start_times = catalog['time_abs'].to_numpy()
#     #     labels = np.isin(data['time_abs'].to_numpy(), quake_start_times).astype(int)
#     return labels

# # Apply labeling
# labels = label_data(data, catalog)

In [6]:
# def label_data(data, catalog):
#     # Convert catalog time_rel to a numpy array for fast lookups
#     quake_start_times = data['time_rel'].to_numpy()

#     # Create an array of labels
#     labels_rel = np.isin(catalog['time_rel'].to_numpy(), quake_start_times).astype(int)
    
#     # Get the indexes of every 1 in the labels array
#     indexes_of_ones = np.where(labels_rel == 1)[0]
#     return data.iloc[indexes_of_ones]

#     #return indexes_of_ones

# # Apply labeling and get indexes
# indexes = label_data(data, catalog)
# print(indexes)

In [7]:
# import numpy as np

# def label_data(data, catalog):
#     # Convert catalog time_rel to a numpy array for fast lookups
#     quake_start_times = catalog['time_rel'].to_numpy()

#     # Create an array of labels
#     labels_rel = np.isin(data['time_rel'].to_numpy(), quake_start_times).astype(int)
    
#     return labels_rel

# # Apply labeling
# labels = label_data(data, catalog)

# # Print the values corresponding to the indices where labels are 1
# matching_rows = data[labels == 1]
# print(matching_rows)

# # Calculate and print the quake start count
# quake_start_count = np.sum(labels)
# print(quake_start_count)

In [8]:
quake_start_times_rel = catalog['time_rel'].to_numpy()
possible_quake_start_times_rel = data['time_rel'].to_numpy()
quake_start_times_abs = catalog['time_abs'].to_numpy()
possible_quake_start_times_abs = data['time_abs'].to_numpy()

def label_data(data, catalog):  
    # labels_rel = np.isin(possible_quake_start_times_rel, quake_start_times_rel )
    # labels_abs = np.isin(possible_quake_start_times_abs, quake_start_times_abs)
    # matching_values_rel = possible_quake_start_times_rel[labels_rel]
    # matching_values_abs = possible_quake_start_times_abs[labels_abs]
    # indexes_of_ones = np.where(labels_rel == 1)[0]

    # Create a boolean mask for matching time_rel
    mask_rel = data['time_rel'].isin(catalog['time_rel'])
    # Filter data and catalog based on the mask
    filtered_data = data[mask_rel]
    filtered_catalog = catalog[catalog['time_rel'].isin(filtered_data['time_rel'])]
    # Create a boolean mask for matching time_abs in the filtered data
    mask_abs = filtered_data.apply(lambda row: row['time_abs'] in filtered_catalog[filtered_catalog['time_rel'] == row['time_rel']]['time_abs'].values, axis=1)
    # Initialize labels array with zeros
    labels = np.zeros(len(data), dtype=int)
    # Set labels to 1 where both masks are True
    labels[mask_rel[mask_rel].index[mask_abs]] = 1
    
    return labels




# Apply labeling
labels = label_data(data, catalog)
print(labels)
# Print the values corresponding to the 7 matches
matching_rows = data[labels == 1]
print(matching_rows)




[0 0 0 ... 0 0 0]
       time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_rel(sec)  velocity(m/s)  \
486938     1970-01-19T20:25:00.740472   73500.075472   9.004942e-10   
486939     1970-01-19T20:25:00.891415   73500.226415   4.575971e-10   
486940     1970-01-19T20:25:01.042358   73500.377358  -4.284551e-10   
486941     1970-01-19T20:25:01.193302   73500.528302  -1.209429e-09   
486942     1970-01-19T20:25:01.344245   73500.679245  -1.350565e-09   
486943     1970-01-19T20:25:01.495189   73500.830189  -6.886146e-10   
486944     1970-01-19T20:25:01.646132   73500.981132   4.306897e-10   

                          time_abs  time_rel      velocity  
486938  1970-01-19T20:25:00.000000     73500  9.004942e-10  
486939  1970-01-19T20:25:00.000000     73500  4.575971e-10  
486940  1970-01-19T20:25:00.000000     73500 -4.284551e-10  
486941  1970-01-19T20:25:00.000000     73500 -1.209429e-09  
486942  1970-01-19T20:25:00.000000     73500 -1.350565e-09  
486943  1970-01-19T20:25:00.000000     73500 -6

In [9]:
quake_start_count = np.sum(labels)
print(quake_start_count)

7
